In [1]:
# bne_llm_prompt_experiments.ipynb

"""Purpose

This notebook connects a Large Language Model (LLM) with the Brisbane battery-recycling MILP model.

- Input: natural-language descriptions of disruption scenarios in the Brisbane Li-ion battery recycling network.
- Step 1 (LLM): the language model converts each description into a structured JSON scenario
  (fields: `scenario_id`, `summary_zh`, `affected_center`, `capacity_factor`, `duration_days`, `unmet_penalty`).
- Step 2 (OR): the JSON scenario is fed into the Gurobi MILP model, which computes optimal flows, total cost,
  unmet demand and facility utilization.
- Step 3 (Comparison): LLM-generated scenarios are compared against engineered scenarios from
  `bne_llm_or_experiment.ipynb` in terms of cost and resilience.

This notebook demonstrates how **LLMs + OR** can support faster and more systematic disruption analysis
for the Brisbane Li-ion battery recycling supply chain."""



'Purpose\n\nThis notebook connects a Large Language Model (LLM) with the Brisbane battery-recycling MILP model.\n\n- Input: natural-language descriptions of disruption scenarios in the Brisbane Li-ion battery recycling network.\n- Step 1 (LLM): the language model converts each description into a structured JSON scenario\n  (fields: `scenario_id`, `summary_zh`, `affected_center`, `capacity_factor`, `duration_days`, `unmet_penalty`).\n- Step 2 (OR): the JSON scenario is fed into the Gurobi MILP model, which computes optimal flows, total cost,\n  unmet demand and facility utilization.\n- Step 3 (Comparison): LLM-generated scenarios are compared against engineered scenarios from\n  `bne_llm_or_experiment.ipynb` in terms of cost and resilience.\n\nThis notebook demonstrates how **LLMs + OR** can support faster and more systematic disruption analysis\nfor the Brisbane Li-ion battery recycling supply chain.'

In [2]:
# === Cell 1：读取四件套 + 第二个实验结果，并对齐索引 ===
from pathlib import Path
import json
import pandas as pd

# === 1) 项目 & 数据路径 ===
PROJECT_ROOT = Path(r"D:\Projects\llm_or_project\Notebook1")
DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("PROCESSED_DIR:", PROCESSED_DIR)

# 一个小工具：自动在 processed 里找同名的 csv/xls/xlsx
def read_table(basename, index_col=None):
    exts_readers = [
        (".csv", pd.read_csv),
        (".xls", pd.read_excel),
        (".xlsx", pd.read_excel),
    ]
    for ext, reader in exts_readers:
        path = PROCESSED_DIR / f"{basename}{ext}"
        if path.exists():
            print(f"[INFO] Reading {path.name}")
            return reader(path, index_col=index_col)
    # 如果都没找到，就抛错提醒
    raise FileNotFoundError(
        f"Cannot find file for '{basename}' with extensions .csv/.xls/.xlsx "
        f"in {PROCESSED_DIR}"
    )

# === 2) 读取“四件套” ===
nodes_df = read_table("nodes")           # nodes.csv / nodes.xls ...
facilities_df = read_table("facilities")
# 距离矩阵：之前在实验 2 中是以 node_id 作为 index 保存的，这里先把第一列作为索引读入
dist_df = read_table("dist", index_col=0)

# === 3) 关键：把索引设成和 bne_llm_or_experiment.ipynb 中一致 ===

# 节点表：索引 = node_id
if "node_id" in nodes_df.columns:
    nodes_df = nodes_df.set_index("node_id")

# 设施表：索引 = facility_id
if "facility_id" in facilities_df.columns:
    facilities_df = facilities_df.set_index("facility_id")

# 距离矩阵：
#   - 行索引 = node_id
#   - 列 = 各 facility_id，且顺序与 facilities_df.index 一致
# 如果 dist_df 里还有名为 node_id 的列，就再设一次索引；否则默认 index 已经是 node_id
if "node_id" in dist_df.columns:
    dist_df = dist_df.set_index("node_id")

# 只保留在设施列表中的列，并按设施顺序重新排列
dist_df = dist_df[facilities_df.index]

# === 4) 场景定义 JSON（来自 brsb_data_generator 实验 1） ===
scenarios_path = PROCESSED_DIR / "scenarios.json"
if not scenarios_path.exists():
    raise FileNotFoundError(f"scenarios.json not found at {scenarios_path}")

with open(scenarios_path, "r", encoding="utf-8") as f:
    scenarios_json = json.load(f)

# === 5) 读取第二个实验的汇总表（实验 2 的结果） ===
exp_summary_path = PROCESSED_DIR / "tbl_experiment_summary_all_scenarios.csv"
if not exp_summary_path.exists():
    raise FileNotFoundError(
        f"tbl_experiment_summary_all_scenarios.csv not found at {exp_summary_path}\n"
        "请确认你已经在 bne_llm_or_experiment.ipynb 里跑完 Cell11 并导出该文件。"
    )

exp_summary_all = pd.read_csv(exp_summary_path)

# === 6) 打印检查信息 ===
print("\n--- Loaded data shapes ---")
print("nodes_df:", nodes_df.shape, " index example:", list(nodes_df.index)[:3])
print("facilities_df:", facilities_df.shape, " index example:", list(facilities_df.index)[:3])
print("dist_df:", dist_df.shape,
      " row index example:", list(dist_df.index)[:3],
      " columns:", list(dist_df.columns))
print("scenarios in scenarios.json:", len(scenarios_json))
print("exp_summary_all:", exp_summary_all.shape)


PROJECT_ROOT: D:\Projects\llm_or_project\Notebook1
PROCESSED_DIR: D:\Projects\llm_or_project\Notebook1\data\processed
[INFO] Reading nodes.csv
[INFO] Reading facilities.csv
[INFO] Reading dist.csv

--- Loaded data shapes ---
nodes_df: (5, 9)  index example: ['N001', 'N002', 'N003']
facilities_df: (3, 12)  index example: ['F01', 'F02', 'F03']
dist_df: (5, 3)  row index example: ['N001', 'N002', 'N003']  columns: ['F01', 'F02', 'F03']
scenarios in scenarios.json: 4
exp_summary_all: (24, 13)


In [3]:
# ============= Cell2_const：等级结构 & 全局参数（与第二实验完全对齐） =============

import pandas as pd

# 小检查：看一下当前数据结构是否和第二个实验一致
print("[DEBUG] nodes_df.columns =", list(nodes_df.columns))
print("[DEBUG] facilities_df.columns =", list(facilities_df.columns))
print("[DEBUG] dist_df.shape    =", dist_df.shape)

# 约定三个等级：A / B / C
GRADES = ["A", "B", "C"]

# 1) 节点需求列名（来自 brsb_data_generator.ipynb）
#    注意：这里使用的列名必须和 nodes_df 里的列名一致
GRADE_DEMAND_COL = {
    "A": "grade_A_t",
    "B": "grade_B_t",
    "C": "grade_C_t",
}

# 2) 设施容量列名
GRADE_CAP_COL = {
    "A": "cap_A_t",
    "B": "cap_B_t",
    "C": "cap_C_t",
}

# 3) 等级接受标记（0/1）
ACCEPT_COL = {
    "A": "accept_A",
    "B": "accept_B",
    "C": "accept_C",
}

# 4) 运输成本参数：每吨·公里的成本（可以在敏感性分析中调整）
TRANSPORT_COST_PER_TKM = {
    "A": 1.0,
    "B": 1.0,
    "C": 1.1,   # 假设危险性更高，稍贵一点
}

# 5) 把距离单位从 m 转成 km（和第二个实验保持完全一致）
#    这里既保留 dist_km_df，又覆盖 dist_df，方便后面 solve_bne_model 直接用 dist_df（km）
dist_km_df = dist_df.copy().astype(float) / 1000.0
dist_df = dist_km_df

print("[INFO] 已将 dist_df 从米转换为公里，示例值：")
print(dist_df.iloc[:3, :3])

# 6) 计算总需求 & 总容量，检查模型是否可行（baseline 视角）
summary_rows = []
for g in GRADES:
    demand_col = GRADE_DEMAND_COL[g]
    cap_col    = GRADE_CAP_COL[g]

    total_demand = nodes_df[demand_col].sum()
    total_capacity = facilities_df[cap_col].sum()

    summary_rows.append({
        "grade": g,
        "total_demand_t": total_demand,
        "total_capacity_t": total_capacity,
        "capacity_minus_demand": total_capacity - total_demand,
    })

system_summary = pd.DataFrame(summary_rows)

print("\n=== System demand vs capacity (baseline, same as Experiment 2) ===")
display(system_summary)


[DEBUG] nodes_df.columns = ['SA2_CODE', 'SA2_NAME', 'LAT', 'LON', 'POP', 'recycled_tonnes_total', 'grade_A_t', 'grade_B_t', 'grade_C_t']
[DEBUG] facilities_df.columns = ['name', 'TYPE', 'LAT', 'LON', 'accept_A', 'accept_B', 'accept_C', 'capacity_total_t', 'cap_A_t', 'cap_B_t', 'cap_C_t', 'fixed_cost']
[DEBUG] dist_df.shape    = (5, 3)
[INFO] 已将 dist_df 从米转换为公里，示例值：
              F01       F02       F03
node_id                              
N001     1.783800  4.174463  5.351186
N002     4.174432  3.794821  6.775914
N003     3.793614  6.775932  6.066695

=== System demand vs capacity (baseline, same as Experiment 2) ===


,grade,total_demand_t,total_capacity_t,capacity_minus_demand
0,A,227.670443,84.0,-143.670443
1,B,81.310872,20.0,-61.310872
2,C,16.262174,2.5,-13.762174


In [4]:
# === Cell 2: 准备基线 & 工程化场景的摘要（给 LLM 当上下文） ===

# 基线场景
baseline_row = exp_summary_all[exp_summary_all["scenario_type"] == "baseline"].iloc[0]

# 工程化场景（两条：F01_40pct_5days, F02_80pct_10days）
eng_df = exp_summary_all[exp_summary_all["scenario_type"] == "engineered_disruption"].copy()

# 方便根据 center_id 查名字
# 假设 facilities_df 的 index 就是 'facility_id'（F01/F02/...），列里有 'name'
FAC_NAME_COL = "name"   # 如果你列名不同，在这里改

def get_center_name(center_id):
    try:
        return str(facilities_df.loc[center_id, FAC_NAME_COL])
    except Exception:
        return center_id

# 为每个受影响中心整理一份“LLM 上下文说明”
center_context = {}

for _, row in eng_df.iterrows():
    center_id = row["affected_center"]
    center_name = get_center_name(center_id)
    
    ctx = {
        "center_id": center_id,
        "center_name": center_name,
        "eng_scenario_id": row["scenario_id"],
        "eng_summary_zh": row["summary_zh"],
        "eng_capacity_factor": row["capacity_factor"],
        "eng_duration_days": row["duration_days"],
        "eng_total_cost": row["total_cost"],
        "eng_total_unmet_t": row["total_unmet_t"],
    }
    center_context[center_id] = ctx

print("Baseline summary:")
print(baseline_row[["scenario_id", "total_cost", "total_unmet_t"]])
print("\nEngineered disruption scenarios:")
display(eng_df[["scenario_id", "affected_center", "summary_zh",
                "capacity_factor", "duration_days",
                "total_cost", "total_unmet_t"]])

print("\ncenter_context keys:", list(center_context.keys()))


Baseline summary:
scenario_id                BASE
total_cost       2353128.179983
total_unmet_t         218.74349
Name: 0, dtype: object

Engineered disruption scenarios:


,scenario_id,affected_center,summary_zh,capacity_factor,duration_days,total_cost,total_unmet_t
1,F02_80pct_10days,F02,场景2：设施 Demo Remanufacturing Plant 1（F02）计划维护，未...,0.8,10,2.394917e+06,224.44349
2,F03_60pct_7days,F03,场景3：设施 Demo Second-life Facility 1（F03）位于偏远区域，...,0.6,7,2.465128e+06,229.94349
3,F01_40pct_5days,F01,场景1：设施 Demo Recycling Center 1（F01）发生故障，未来5天处理...,0.4,5,2.599347e+06,248.74349
20,ENG_F01_40pct_5days,F01,F01 回收中心发生严重故障，未来 5 天容量仅为平时的 40%。,0.4,5,2.487547e+06,248.74349
21,ENG_F02_80pct_10days,F02,F02 再制造工厂出现中度故障，未来 10 天容量降为 80%。,0.8,10,3.366803e+06,224.44349
22,ENG_F01_120pct_14days,F01,F01 回收中心通过加班与临时工扩容，未来 14 天容量提升到 120%。,1.2,14,2.087619e+06,208.74349
23,ENG_F02_60pct_3days,F02,F02 再制造工厂因短期检修，未来 3 天容量降为 60%。,0.6,3,3.452288e+06,230.14349



center_context keys: ['F02', 'F03', 'F01']


In [5]:
# === Cell3_solve_model：定义 Gurobi 模型 + 基准场景 baseline ===
from gurobipy import Model, GRB, quicksum
import pandas as pd

def solve_bne_model(capacity_factor=None, unmet_penalty=10000, verbose=True):
    """
    Solve Brisbane battery-network model.

    Parameters
    ----------
    capacity_factor :
        - None        → 所有设施容量系数 = 1.0
        - float/int   → 所有设施容量统一乘以这个系数
        - dict        → {facility_id: 系数}，未给出的设施默认 1.0
    unmet_penalty : float
        每吨未满足需求的惩罚成本（统一对 A/B/C 适用）
    verbose : bool
        是否打印 Gurobi 日志

    Returns
    -------
    result : dict
        {
          "model":       m,             # Gurobi model 对象
          "status":      m.status,      # Gurobi 状态码
          "total_cost":  m.objVal,      # 目标值
          "total_unmet": total_unmet,   # 总未满足需求 (t)
          "util_df":     util_df        # 各设施各等级利用率表
        }
    """

    # -------- 0. 把 capacity_factor 统一变成 {facility_id: factor} 的 dict --------
    fac_ids = list(facilities_df.index)

    if capacity_factor is None:
        cap_factor = {fid: 1.0 for fid in fac_ids}

    elif isinstance(capacity_factor, (int, float)):
        cf = float(capacity_factor)
        cap_factor = {fid: cf for fid in fac_ids}

    elif isinstance(capacity_factor, dict):
        cap_factor = {}
        for fid in fac_ids:
            cap_factor[fid] = float(capacity_factor.get(fid, 1.0))
    else:
        raise TypeError(
            f"capacity_factor must be None, float/int or dict, got {type(capacity_factor)}"
        )

    # -------- 1. 准备数据 --------
    node_ids = list(nodes_df.index)
    grades = GRADES  # ["A", "B", "C"]

    # 需求 (t)
    demand = {
        (i, g): float(nodes_df.loc[i, GRADE_DEMAND_COL[g]])
        for i in node_ids
        for g in grades
    }

    # 设施容量 (t)（含接受约束 + capacity_factor）
    cap = {}
    for j in fac_ids:
        for g in grades:
            base_cap = float(facilities_df.loc[j, GRADE_CAP_COL[g]])
            accept   = int(facilities_df.loc[j, ACCEPT_COL[g]])
            eff_cap  = base_cap * cap_factor[j] * accept  # 不接受该等级时 accept=0
            cap[(j, g)] = eff_cap

    # 距离 (km)  dist_df: index=node_id, columns=facility_id
    # 这里保持和第二个实验一致：dist_df 里已经是 km（或者你在数据生成时就处理过）
    dist_km = {
        (i, j): float(dist_df.loc[i, j])
        for i in node_ids
        for j in fac_ids
    }

    # 吨公里成本
    trans_cost = TRANSPORT_COST_PER_TKM  # dict: {"A":..., "B":..., "C":...}

    # -------- 2. 建立 Gurobi 模型 --------
    m = Model("bne_llm_or")
    m.Params.OutputFlag = 1 if verbose else 0

    # 决策变量：x[i,j,g] 运输量 (t)，u[i,g] 未满足需求 (t)
    x = {}
    for i in node_ids:
        for j in fac_ids:
            for g in grades:
                x[(i, j, g)] = m.addVar(lb=0.0, name=f"x_{i}_{j}_{g}")

    u = {}
    for i in node_ids:
        for g in grades:
            u[(i, g)] = m.addVar(lb=0.0, name=f"u_{i}_{g}")

    m.update()

    # -------- 3. 约束 --------
    # (a) 每个节点每个等级：供应 + 未满足 = 需求
    for i in node_ids:
        for g in grades:
            m.addConstr(
                quicksum(x[(i, j, g)] for j in fac_ids) + u[(i, g)] == demand[(i, g)],
                name=f"demand_{i}_{g}"
            )

    # (b) 每个设施每个等级：流入 ≤ 有效容量
    for j in fac_ids:
        for g in grades:
            if cap[(j, g)] > 0:
                m.addConstr(
                    quicksum(x[(i, j, g)] for i in node_ids) <= cap[(j, g)],
                    name=f"cap_{j}_{g}"
                )
            else:
                # 如果容量为 0，就强制 x = 0
                for i in node_ids:
                    m.addConstr(
                        x[(i, j, g)] == 0.0,
                        name=f"cap_zero_{i}_{j}_{g}"
                    )

    # -------- 4. 目标函数：运输成本 + 未满足惩罚 --------
    transport_term = quicksum(
        x[(i, j, g)] * dist_km[(i, j)] * trans_cost[g]
        for i in node_ids
        for j in fac_ids
        for g in grades
    )

    unmet_term = unmet_penalty * quicksum(
        u[(i, g)] for i in node_ids for g in grades
    )

    m.setObjective(transport_term + unmet_term, GRB.MINIMIZE)

    # -------- 5. 求解 --------
    m.optimize()

    status = m.status
    total_cost = float(m.ObjVal) if status == GRB.OPTIMAL else None

    # 总未满足需求
    total_unmet = 0.0
    if status == GRB.OPTIMAL:
        total_unmet = sum(u[(i, g)].X for i in node_ids for g in grades)

    # -------- 6. 生成利用率表 util_df --------
    util_rows = []
    if status == GRB.OPTIMAL:
        for j in fac_ids:
            for g in grades:
                flow = sum(x[(i, j, g)].X for i in node_ids)
                cap_t = cap[(j, g)]
                util = flow / cap_t if cap_t > 0 else 0.0
                util_rows.append({
                    "facility_id": j,
                    "grade":      g,
                    "flow_t":     flow,
                    "cap_t":      cap_t,
                    "utilization": util,
                })

    util_df = pd.DataFrame(util_rows)

    result = {
        "model":       m,
        "status":      status,
        "total_cost":  total_cost,
        "total_unmet": total_unmet,
        "util_df":     util_df,
    }

    return result


# ===== 可选：在同一个 Cell 里跑一次 baseline，方便后面复用 =====
print("=== Solving baseline Brisbane battery-network model (Cell3) ===")

unmet_penalty_base = 10000  # 和第二个实验保持一致

baseline_res = solve_bne_model(
    capacity_factor=1.0,
    unmet_penalty=unmet_penalty_base,
    verbose=False,
)

if baseline_res["status"] == GRB.OPTIMAL:
    baseline_cost  = baseline_res["total_cost"]
    baseline_unmet = baseline_res["total_unmet"]
    print(f"Baseline total cost: {baseline_cost:,.2f}")
    print(f"Baseline unmet demand (t): {baseline_unmet:.4f}")
else:
    print("Baseline model not optimal. Status =", baseline_res["status"])


=== Solving baseline Brisbane battery-network model (Cell3) ===
Set parameter Username
Set parameter LicenseID to value 2737110
Baseline total cost: 2,187,600.59
Baseline unmet demand (t): 218.7435


In [6]:
# === Cell 3: 定义 LLM 场景 schema & 校验/规范化函数 ===
import math

SCENARIO_FIELDS = [
    "scenario_id",
    "summary_zh",
    "affected_center",
    "capacity_factor",
    "duration_days",
    "unmet_penalty",
]

DEFAULT_UNMET_PENALTY = int(baseline_row["unmet_penalty"]) if "unmet_penalty" in baseline_row else 10000

def normalize_llm_scenario(raw_dict, center_id_hint=None):
    """
    把 LLM 返回的 dict 规范化成我们需要的字段和类型。
    若缺字段则补默认值，类型不对时尝试转换。
    """
    scen = {}
    
    # 1) scenario_id
    scen_id = raw_dict.get("scenario_id") or raw_dict.get("name") or ""
    scen_id = str(scen_id).strip()
    if not scen_id and center_id_hint:
        scen_id = f"LLM_case_{center_id_hint}"
    scen["scenario_id"] = scen_id

    # 2) summary_zh
    summary = raw_dict.get("summary_zh") or raw_dict.get("summary") or ""
    scen["summary_zh"] = str(summary).strip()

    # 3) affected_center
    center = raw_dict.get("affected_center") or center_id_hint or ""
    scen["affected_center"] = str(center).strip()

    # 4) capacity_factor
    cf = raw_dict.get("capacity_factor", 1.0)
    try:
        cf = float(cf)
    except Exception:
        cf = 1.0
    # 限制在 0~1 之间
    cf = max(0.0, min(1.0, cf))
    scen["capacity_factor"] = cf

    # 5) duration_days
    dur = raw_dict.get("duration_days", 0)
    try:
        dur = int(round(float(dur)))
    except Exception:
        dur = 0
    dur = max(0, dur)
    scen["duration_days"] = dur

    # 6) unmet_penalty
    p = raw_dict.get("unmet_penalty", DEFAULT_UNMET_PENALTY)
    try:
        p = int(round(float(p)))
    except Exception:
        p = DEFAULT_UNMET_PENALTY
    scen["unmet_penalty"] = p

    # 额外 meta，可视需要加入
    scen["scenario_type"] = "LLM_generated"

    return scen

print("DEFAULT_UNMET_PENALTY:", DEFAULT_UNMET_PENALTY)


DEFAULT_UNMET_PENALTY: 10000


In [7]:
# === Cell 4: Prompt 模板 ===

PROMPT_TEMPLATE = """你是一名供应链与运筹优化专家，正在分析布里斯班锂电池回收网络。

[基线场景 Base]
- 所有回收中心正常运行。
- 总成本：{base_cost_m:.2f} 百万 AUD
- 未满足需求：{base_unmet:.1f} 吨。

[工程化扰动场景：{eng_scenario_id}]
- 受影响中心：{center_id}（{center_name}）
- 场景描述：{eng_summary}
- 容量系数 capacity_factor：{eng_capacity_factor:.1f}
- 持续时间 duration_days：{eng_duration_days} 天
- 总成本：{eng_cost_m:.2f} 百万 AUD
- 未满足需求：{eng_unmet:.1f} 吨。

现在，请你只针对同一个受影响中心 {center_id} 设计一个“LLM 推荐场景”，满足以下要求：
1. 只修改这个中心的 capacity_factor（0~1 之间的小数）和 duration_days（非负整数，单位：天）。
2. unmet_penalty 统一保持为 {unmet_penalty} （每吨未满足需求的惩罚成本）。
3. 目标是在“总成本”和“未满足需求”之间做如下权衡：{goal_text}
4. 请直接给出一个 JSON 对象，不要额外解释。

JSON 字段要求：

- "scenario_id": 字符串，建议以 "LLM_case_{center_id}_" 开头。
- "summary_zh": 一句简短的中文场景描述。
- "affected_center": 字符串，比如 "F01"。
- "capacity_factor": 0~1 之间的小数，比如 0.6。
- "duration_days": 非负整数，比如 7。
- "unmet_penalty": 整数，比如 10000。

只输出 JSON，对 JSON 外的任何文本（包括反引号、说明文字）都不要输出。
"""

def build_prompt_for_center(center_id, goal_text):
    ctx = center_context[center_id]
    base_cost = float(baseline_row["total_cost"])
    base_unmet = float(baseline_row["total_unmet_t"])
    eng_cost = float(ctx["eng_total_cost"])
    eng_unmet = float(ctx["eng_total_unmet_t"])
    
    prompt = PROMPT_TEMPLATE.format(
        base_cost_m=base_cost / 1e6,
        base_unmet=base_unmet,
        eng_scenario_id=ctx["eng_scenario_id"],
        center_id=center_id,
        center_name=ctx["center_name"],
        eng_summary=ctx["eng_summary_zh"],
        eng_capacity_factor=ctx["eng_capacity_factor"],
        eng_duration_days=int(ctx["eng_duration_days"]),
        eng_cost_m=eng_cost / 1e6,
        eng_unmet=eng_unmet,
        unmet_penalty=DEFAULT_UNMET_PENALTY,
        goal_text=goal_text,
    )
    return prompt

# 小测试：打印一个 prompt 看看
test_prompt = build_prompt_for_center("F01", "在总成本不增加太多的前提下，尽量减少未满足需求。")
print(test_prompt[:600], "...\n\n[Prompt length]", len(test_prompt))


你是一名供应链与运筹优化专家，正在分析布里斯班锂电池回收网络。

[基线场景 Base]
- 所有回收中心正常运行。
- 总成本：2.35 百万 AUD
- 未满足需求：218.7 吨。

[工程化扰动场景：ENG_F01_120pct_14days]
- 受影响中心：F01（Demo Recycling Center 1）
- 场景描述：F01 回收中心通过加班与临时工扩容，未来 14 天容量提升到 120%。
- 容量系数 capacity_factor：1.2
- 持续时间 duration_days：14 天
- 总成本：2.09 百万 AUD
- 未满足需求：208.7 吨。

现在，请你只针对同一个受影响中心 F01 设计一个“LLM 推荐场景”，满足以下要求：
1. 只修改这个中心的 capacity_factor（0~1 之间的小数）和 duration_days（非负整数，单位：天）。
2. unmet_penalty 统一保持为 10000 （每吨未满足需求的惩罚成本）。
3. 目标是在“总成本”和“未满足需求”之间做如下权衡：在总成本不增加太多的前提下，尽量减少未满足需求。
4. 请直接给出一个 JSON 对象，不要额外解释。

JSON 字段要求：

- "scenario_id": 字符串，建议以 "LLM_case_F01_" 开头。
- "summar ...

[Prompt length] 798


In [8]:
# ===== Cell 5: LLM 调用封装（真 API + 可选假数据） =====
import os
import json
from typing import Dict, Any

try:
    # 新版 openai SDK
    from openai import OpenAI
    _client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    _USE_RESPONSES_API = True
except Exception:
    # 兼容老版本 openai 库
    import openai
    openai.api_key = os.getenv("OPENAI_API_KEY")
    _client = None
    _USE_RESPONSES_API = False


def _fake_llm_answer() -> str:
    """
    假数据：返回一段“看起来像 LLM 输出”的 JSON 文本，方便本地离线调试。
    你可以根据需要改成任意场景。
    """
    fake_obj = {
        "scenario_id": "F01_40pct_5days",
        "summary_zh": "场景1：拟在 Demo Recycling Center 1 (F01) 发生故障，未来5天处理能力降为40%。",
        "affected_center": "F01",
        "capacity_factor": 0.4,
        "duration_days": 5,
        "unmet_penalty": 10000
    }
    return json.dumps(fake_obj, ensure_ascii=False, indent=2)


def call_llm_scenario(
    prompt: str,
    model: str = "gpt-4.1-mini",
    use_fake: bool = False,
    max_output_tokens: int = 600,
) -> Dict[str, Any]:
    """
    调用 LLM 生成场景说明，或返回假数据。

    Parameters
    ----------
    prompt : str
        给 LLM 的完整提示词（已经包含中文/英文说明）
    model : str
        OpenAI 模型名称（默认 gpt-4.1-mini，可根据你账号权限调整）
    use_fake : bool
        True  → 不真的调用 API，而是返回一段固定 JSON 文本（离线调试用）
        False → 调用 OpenAI API
    max_output_tokens : int
        LLM 最多生成多少 token

    Returns
    -------
    result : dict
        {
          "raw_text": str,        # LLM 原始文本输出
          "response": object|None # OpenAI 返回对象（假数据模式下为 None）
        }
    """
    # 1) 假数据模式：完全不访问网络
    if use_fake:
        raw_text = _fake_llm_answer()
        return {"raw_text": raw_text, "response": None}

    # 2) 真 API 调用
    if _USE_RESPONSES_API:
        # 新版 Responses API
        resp = _client.responses.create(
            model=model,
            input=prompt,
            max_output_tokens=max_output_tokens,
        )
        # 兼容性提取文本：优先用 responses 格式
        try:
            raw_text = resp.output[0].content[0].text
        except Exception:
            raw_text = str(resp)
    else:
        # 老版 Chat Completions 调用（如果你环境里还是旧 SDK）
        resp = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ],
            max_tokens=max_output_tokens,
        )
        try:
            raw_text = resp["choices"][0]["message"]["content"]
        except Exception:
            raw_text = str(resp)

    return {"raw_text": raw_text, "response": resp}


In [9]:
# ============= Cell 6: 从 LLM raw_text 抽出 JSON 并做校验 =============
import json
import re

def _extract_json_block(raw_text: str):
    """
    尝试从 LLM 的输出中抽出第一个 JSON 对象。
    支持三种常见格式：
    1）```json ... ``` 代码块
    2）``` ... ``` 代码块
    3）裸的 { ... } JSON 段落

    返回: Python dict
    如果失败: 抛出 ValueError
    """

    text = raw_text.strip()
    candidates = []

    # 1) 优先找 ```json ... ``` 代码块
    m = re.search(r"```json(.*?)```", text, flags=re.S | re.I)
    if m:
        candidates.append(m.group(1))

    # 2) 再找 ``` ... ``` 代码块
    if not candidates:
        m = re.search(r"```(.*?)```", text, flags=re.S)
        if m:
            candidates.append(m.group(1))

    # 3) 如果前两种都没有，退而求其次：找第一个 {...}
    if not candidates:
        # 非贪婪匹配，尽量抓到第一个成对的大括号
        m = re.search(r"\{.*\}", text, flags=re.S)
        if m:
            candidates.append(m.group(0))

    if not candidates:
        raise ValueError("在 LLM 输出中没有找到 JSON 片段。")

    last_err = None
    for cand in candidates:
        s = cand.strip()
        # 第一次尝试：原样解析
        try:
            return json.loads(s)
        except Exception as e:
            last_err = e
            # 第二次尝试：把单引号替换成双引号（部分模型喜欢用单引号）
            try:
                s_fixed = re.sub(r"'", '"', s)
                return json.loads(s_fixed)
            except Exception as e2:
                last_err = e2
                continue

    raise ValueError(f"找到了 JSON 片段但解析失败，最后一次错误: {last_err}")


def parse_llm_scenario_output(
    raw_text: str,
    default_unmet_penalty: float = 10000.0,
    max_capacity_factor: float = 2.0,
):
    """
    将 LLM 输出的 raw_text → 解析成标准化的 scenario_dict，并做校验。

    期望 LLM 输出中包含的 JSON 字段（区分大小写不严格，会自动转小写）:
      - scenario_id        (str)   场景 ID
      - summary_zh         (str)   中文摘要（可选）
      - affected_center    (str)   受影响的设施 ID，例如 "F01"
      - capacity_factor    (float) 容量缩放系数，例如 0.4 / 0.8 / 1.2
      - duration_days      (int)   影响持续天数
      - unmet_penalty      (float) 每吨未满足需求的惩罚成本（可选）

    返回:
      scenario_dict = {
        "scenario_id": ...,
        "summary_zh": ...,
        "affected_center": ...,
        "capacity_factor": ...,
        "duration_days": ...,
        "unmet_penalty": ...,
        "raw_json": 原始 JSON 对象,
      }
    """

    # 1) 抽出 JSON
    try:
        raw_obj = _extract_json_block(raw_text)
    except Exception as e:
        raise RuntimeError(
            "从 LLM 输出中抽取 JSON 失败: "
            f"{e}\n\n=== LLM raw_text (前 1000 字符) ===\n{raw_text[:1000]}"
        )

    if not isinstance(raw_obj, dict):
        raise RuntimeError(f"解析得到的 JSON 不是对象(dict)，而是: {type(raw_obj)}")

    # 2) 做一个“大小写不敏感”的视图
    lower_map = {str(k).lower(): v for k, v in raw_obj.items()}

    def get_any(*names, default=None, required=False):
        """在 raw_obj / lower_map 里找一组候选字段名"""
        for n in names:
            if n in raw_obj:
                return raw_obj[n]
            ln = str(n).lower()
            if ln in lower_map:
                return lower_map[ln]
        if required:
            raise RuntimeError(f"LLM JSON 缺少必需字段: {names}")
        return default

    # 3) 逐个字段解析 / 校验
    scenario_id = get_any("scenario_id", "id", required=True)
    summary_zh = get_any("summary_zh", "summary", "description", default="")

    affected_center = get_any(
        "affected_center", "center", "facility", "facility_id", required=True
    )
    affected_center = str(affected_center).strip()

    # 如果前面 Cell1 已经读过 facilities_df，可以在这里做一个“设施存在性检查”
    if "facilities_df" in globals():
        # 确保索引都是字符串，避免 0/1/2 vs "F01" 这种类型不匹配
        valid_centers = [str(x) for x in facilities_df.index]
        if affected_center not in valid_centers:
            raise RuntimeError(
                f"LLM 给出的 affected_center='{affected_center}' 不在已知设施列表中: {valid_centers}"
            )

    # capacity_factor
    cf_raw = get_any("capacity_factor", "capacity_factor_multiplier", default=1.0)
    try:
        capacity_factor = float(cf_raw)
    except Exception:
        raise RuntimeError(f"capacity_factor 无法转换为 float: {cf_raw!r}")

    if capacity_factor <= 0 or capacity_factor > max_capacity_factor:
        raise RuntimeError(
            f"capacity_factor={capacity_factor} 超出合理范围 (0, {max_capacity_factor}]，"
            "请检查 LLM 输出。"
        )

    # duration_days
    dur_raw = get_any("duration_days", "duration", default=7)
    try:
        duration_days = int(dur_raw)
    except Exception:
        raise RuntimeError(f"duration_days 无法转换为 int: {dur_raw!r}")
    if duration_days <= 0:
        raise RuntimeError(f"duration_days 必须为正整数，当前为: {duration_days}")

    # ===== unmet_penalty（关键修改：回退到默认值，不再抛错） =====
    unmet_raw = get_any("unmet_penalty", "unmet_penalty_per_tonne", default=None)

    if unmet_raw is None:
        # LLM 没给这个字段，直接用默认值
        unmet_penalty = float(default_unmet_penalty)
    else:
        # LLM 给了，就尝试转成 float；失败或非正数时，打印 warning 并回退默认值
        try:
            unmet_penalty = float(unmet_raw)
        except Exception:
            print(
                f"[WARN] LLM 给出的 unmet_penalty={unmet_raw!r} 无法转换为 float，"
                f"自动回退为默认值 {default_unmet_penalty}。"
            )
            unmet_penalty = float(default_unmet_penalty)

    if unmet_penalty <= 0:
        print(
            f"[WARN] LLM 给出的 unmet_penalty={unmet_penalty} 非正数，"
            f"自动回退为默认值 {default_unmet_penalty}。"
        )
        unmet_penalty = float(default_unmet_penalty)
    # ===== unmet_penalty 处理结束 =====

    scenario_dict = {
        "scenario_id": str(scenario_id),
        "summary_zh": str(summary_zh),
        "affected_center": affected_center,
        "capacity_factor": capacity_factor,
        "duration_days": duration_days,
        "unmet_penalty": unmet_penalty,
        "raw_json": raw_obj,  # 方便以后 debug / 回溯
    }

    return scenario_dict


# ---- 可选：如果你已经在 Cell5 调过一次 LLM，可以在这里测试一下 ----
if "demo_llm_result" in globals():
    print("\n[DEBUG] 尝试从 demo_llm_result['raw_text'] 解析场景 ...")
    scen = parse_llm_scenario_output(demo_llm_result["raw_text"])
    print("解析成功，得到 scenario_dict：")
    display(scen)
else:
    print("Cell 6 已定义 parse_llm_scenario_output()，可在后续 Cell 中对 LLM 输出调用。")


Cell 6 已定义 parse_llm_scenario_output()，可在后续 Cell 中对 LLM 输出调用。


In [10]:
# === 辅助函数：从 scenario_dict 调用 Gurobi 并返回一行结果 ===
import pandas as pd

def run_scenario_from_dict(
    name: str,
    scenario_dict: dict,
    baseline_cost: float | None = None,
    verbose: bool = False,
):
    """
    根据一个 scenario_dict 组装 capacity_factor/unmet_penalty，
    调用 solve_bne_model 求解，并返回：
      - 一行 summary（Series）
      - 原始 res（solve_bne_model 的返回 dict）

    参数
    ----
    name : str
        场景名称（用于表格和打印）
    scenario_dict : dict
        来自工程化设计或 LLM 的场景字典，字段约定：
        - "scenario_id"
        - "summary_zh"
        - "affected_center" : 例如 "F01" / "F02"
        - "capacity_factor" : float, 0~1
        - "duration_days"   : int，可选
        - "unmet_penalty"   : float, 可选，缺省 10000
    baseline_cost : float or None
        baseline 的总成本，用来计算 extra_cost_vs_baseline；
        如果为 None，则该列记为 None。
    verbose : bool
        是否打印 Gurobi 求解日志
    """

    # -------- 1) 从 dict 里安全地取字段 --------
    sid = scenario_dict.get("scenario_id", name)
    summary_zh = scenario_dict.get("summary_zh", "")
    affected_center = scenario_dict.get("affected_center", None)

    # 若 LLM 没给，就用 1.0 作为 capacity_factor
    cap_factor = float(scenario_dict.get("capacity_factor", 1.0))

    # 若没给 duration_days，就设 None，不参与 OR 求解，只用来记在表里
    duration_days = scenario_dict.get("duration_days", None)

    # unmet_penalty：如果缺省，就沿用第二个实验的 10000
    unmet_penalty = float(scenario_dict.get("unmet_penalty", 10000))

    # -------- 2) 组装 capacity_factor 映射（所有设施 = 1.0，受影响中心按给定比例缩放） --------
    fac_ids = list(facilities_df.index)
    cap_factor_dict = {fid: 1.0 for fid in fac_ids}

    if affected_center is not None:
        if affected_center not in fac_ids:
            raise RuntimeError(
                f"场景 {name} 中的 affected_center='{affected_center}' 不在设施列表中：{fac_ids}"
            )
        cap_factor_dict[affected_center] = cap_factor

    # -------- 3) 调用 Gurobi 模型 --------
    res = solve_bne_model(
        capacity_factor=cap_factor_dict,
        unmet_penalty=unmet_penalty,
        verbose=verbose,
    )

    total_cost = res["total_cost"]
    total_unmet = res.get("total_unmet", None)

    # -------- 4) 组装一行 summary（与第二个实验的表结构尽量一致） --------
    extra_cost = None
    if baseline_cost is not None and total_cost is not None:
        extra_cost = total_cost - baseline_cost

    # 计算平均利用率（整个网络所有设施的平均）
    avg_util = None
    util_df = res.get("util_df", None)
    if isinstance(util_df, pd.DataFrame) and "utilization" in util_df.columns:
        avg_util = util_df["utilization"].mean()

    row = {
        "scenario_id": sid,
        "scenario_type": "LLM_generated",
        "summary_zh": summary_zh,
        "affected_center": affected_center,
        "capacity_factor": cap_factor,
        "duration_days": duration_days,
        "unmet_penalty": unmet_penalty,
        "total_cost": total_cost,
        "total_unmet_t": total_unmet,
        "extra_cost_vs_baseline": extra_cost,
        "avg_utilization": avg_util,
    }

    row_series = pd.Series(row, name=sid)
    return row_series, res


In [11]:
# === Cell 6b：预先定义一批 LLM 场景描述（自然语言） ===

LLM_CASE_PROMPTS = { 
    "LLM_case_prompt_1": """
场景A：设施 Demo Recycling Center 1 (F01) 发生故障，未来5天处理能力降为40%。
""",

    "LLM_case_prompt_2": """
场景B：设施 Remanufacturing Plant 1 (F02) 因电力检修，未来10天只能维持60%的处理能力。
请根据该描述，生成一个 JSON 场景对象，字段包括：
"scenario_id", "summary_zh", "affected_center", "capacity_factor", "duration_days", "unmet_penalty"。
unmet_penalty 如果没有特别说明，请使用 15000。
""",

    "LLM_case_prompt_3": """
场景C：F01 的进料道路出现交通拥堵，预计未来7天只能处理平时70%的电池，
其余需求将延迟或外包，未满足需求的惩罚成本可以适当提高到 12000。
同样请返回规范的 JSON 对象，字段同上。
""",
}


In [12]:
# === Cell 7：一次性跑多个 LLM 场景，并与 baseline 统一对比 ===

import json
import re
import pandas as pd

# ---------------------------------------------------------
# 1) 重新定义解析函数：支持 'Demo Recycling Center 1 (F01)' 这类写法
# ---------------------------------------------------------

def _extract_json_block(raw_text: str):
    """从 LLM 输出中抽第一个 JSON 对象（支持 ```json``` / ``` / 裸 { } ）"""
    text = raw_text.strip()
    candidates = []

    # ```json ... ```
    m = re.search(r"```json(.*?)```", text, flags=re.S | re.I)
    if m:
        candidates.append(m.group(1))

    # ``` ... ```
    if not candidates:
        m = re.search(r"```(.*?)```", text, flags=re.S)
        if m:
            candidates.append(m.group(1))

    # 裸 { ... }
    if not candidates:
        m = re.search(r"\{.*\}", text, flags=re.S)
        if m:
            candidates.append(m.group(0))

    if not candidates:
        raise ValueError("在 LLM 输出中没有找到 JSON 片段。")

    last_err = None
    for cand in candidates:
        s = cand.strip()
        try:
            return json.loads(s)
        except Exception as e:
            last_err = e
            try:
                s_fixed = re.sub(r"'", '"', s)
                return json.loads(s_fixed)
            except Exception as e2:
                last_err = e2
                continue

    raise ValueError(f"找到了 JSON 片段但解析失败，最后一次错误: {last_err}")


def parse_llm_scenario_output(
    raw_text: str,
    default_unmet_penalty: float = 10000.0,
    max_capacity_factor: float = 2.0,
):
    """
    把 LLM 的 raw_text 解析成标准化的 scenario_dict，并做校验。
    这里对 affected_center 做了「容错」：
    - 允许 LLM 写 'Demo Recycling Center 1 (F01)'
    - 会自动从中提取 'F01' 和 facilities_df.index 对齐
    """

    # 1) 抽 JSON
    raw_obj = _extract_json_block(raw_text)
    if not isinstance(raw_obj, dict):
        raise RuntimeError(f"解析得到的 JSON 不是对象(dict)，而是: {type(raw_obj)}")

    lower_map = {str(k).lower(): v for k, v in raw_obj.items()}

    def get_any(*names, default=None, required=False):
        for n in names:
            if n in raw_obj:
                return raw_obj[n]
            ln = str(n).lower()
            if ln in lower_map:
                return lower_map[ln]
        if required:
            raise RuntimeError(f"LLM JSON 缺少必需字段: {names}")
        return default

    # 基本字段
    scenario_id = get_any("scenario_id", "id", required=True)
    summary_zh = get_any("summary_zh", "summary", "description", default="")

    affected_center_raw = get_any(
        "affected_center", "center", "facility", "facility_id", required=True
    )
    affected_center_raw = str(affected_center_raw).strip()

    # 关键：这里做「提取 F01 / F02 / F03」的容错
    affected_center = affected_center_raw
    if "facilities_df" in globals():
        valid_centers = [str(x) for x in facilities_df.index]
        if affected_center not in valid_centers:
            m = re.search(r"F\d{2}", affected_center_raw)
            if m:
                ac_extracted = m.group(0)
                if ac_extracted in valid_centers:
                    affected_center = ac_extracted
                else:
                    raise RuntimeError(
                        f"LLM 给出的 affected_center='{affected_center_raw}'，"
                        f"提取出的 '{ac_extracted}' 也不在已知设施列表中: {valid_centers}"
                    )
            else:
                raise RuntimeError(
                    f"LLM 给出的 affected_center='{affected_center_raw}' 不在已知设施列表中: {valid_centers}"
                )

    # capacity_factor
    cf_raw = get_any("capacity_factor", "capacity_factor_multiplier", default=1.0)
    try:
        capacity_factor = float(cf_raw)
    except Exception:
        raise RuntimeError(f"capacity_factor 无法转换为 float: {cf_raw!r}")
    if capacity_factor <= 0 or capacity_factor > max_capacity_factor:
        raise RuntimeError(
            f"capacity_factor={capacity_factor} 超出合理范围 (0, {max_capacity_factor}]"
        )

    # duration_days
    dur_raw = get_any("duration_days", "duration", default=7)
    try:
        duration_days = int(dur_raw)
    except Exception:
        raise RuntimeError(f"duration_days 无法转换为 int: {dur_raw!r}")
    if duration_days <= 0:
        raise RuntimeError(f"duration_days 必须为正整数，当前为: {duration_days}")

    # unmet_penalty
    unmet_raw = get_any("unmet_penalty", "unmet_penalty_per_tonne", default=None)
    if unmet_raw is None:
        unmet_penalty = float(default_unmet_penalty)
    else:
        try:
            unmet_penalty = float(unmet_raw)
        except Exception:
            raise RuntimeError(f"unmet_penalty 无法转换为 float: {unmet_raw!r}")
        if unmet_penalty <= 0:
            raise RuntimeError(
                f"unmet_penalty 必须为正数，当前为: {unmet_penalty}"
            )

    scenario_dict = {
        "scenario_id": str(scenario_id),
        "summary_zh": str(summary_zh),
        "affected_center": affected_center,
        "capacity_factor": capacity_factor,
        "duration_days": duration_days,
        "unmet_penalty": unmet_penalty,
        "raw_json": raw_obj,
    }
    return scenario_dict


# ---------------------------------------------------------
# 2) 准备 baseline 成本（从第二个实验 / baseline_res 里拿）
# ---------------------------------------------------------

if "baseline_cost" not in globals():
    if "exp_summary_all" in globals():
        base_row = exp_summary_all[
            exp_summary_all["scenario_type"] == "baseline"
        ].iloc[0]
        baseline_cost = float(base_row["total_cost"])
        print(
            f">>> baseline_cost 从 exp_summary_all 读取: {baseline_cost:,.2f} AUD"
        )
    elif "baseline_res" in globals():
        baseline_cost = float(baseline_res["total_cost"])
        print(
            f">>> baseline_cost 从 baseline_res 读取: {baseline_cost:,.2f} AUD"
        )
    else:
        raise RuntimeError(
            "找不到 baseline_cost，请确认前面已经跑完第二个实验并得到 baseline 结果。"
        )
else:
    print(f"baseline_cost 已存在: {baseline_cost:,.2f} AUD")

print(f"\nBaseline total cost: {baseline_cost:,.2f} AUD\n")

# ---------------------------------------------------------
# 3) 依次跑所有 LLM_CASE_PROMPTS，并与 baseline 对比
# ---------------------------------------------------------

llm_rows = []
llm_details = {}

for case_id, case_prompt in LLM_CASE_PROMPTS.items():
    print("=" * 70)
    print(f"[{case_id}] 调用 LLM ...\n")

    # 3.1 组装完整 prompt（在你写的 case_prompt 外面加统一说明）
    prompt = (
        "你是一名帮助研究澳大利亚布里斯班地区回收/再制造电池供应链的运筹分析助手。\n"
        "下面是一段关于某个回收/再制造中心发生故障的自然语言描述（中文/英文）：\n\n"
        f"{case_prompt.strip()}\n\n"
        "请根据该描述，生成一个规范的 JSON 场景对象，字段包括：\n"
        '  \"scenario_id\", \"summary_zh\", \"affected_center\", '
        '\"capacity_factor\", \"duration_days\", \"unmet_penalty\"。\n'
        "注意：\n"
        "- affected_center 只能填设施代码，例如 \"F01\" 或 \"F02\"；不要写完整名称。\n"
        "- capacity_factor 必须是 0~2 之间的小数，例如 0.4 / 0.7 / 1.2。\n"
        "- 如果没有特别说明，unmet_penalty 请使用 10000。\n"
        "只输出 JSON，不要输出额外的解释文字。\n"
    )

    # 3.2 调 LLM
    llm_res = call_llm_scenario(prompt, use_fake=False)
    raw_text = llm_res["raw_text"]

    print("\n--- LLM 原始输出（前 400 字符）---\n")
    print(raw_text[:400] + "...\n")

    # 3.3 解析成 scenario_dict（用上面修正过的解析函数）
    scen = parse_llm_scenario_output(
        raw_text,
        default_unmet_penalty=10000.0,
        max_capacity_factor=2.0,
    )
    if not scen.get("scenario_id"):
        scen["scenario_id"] = case_id

    print("--- 从 LLM 输出解析得到的 scenario_dict ---")
    display(scen)

    # 3.4 调 OR 模型并与 baseline 对比（用前面定义的 run_scenario_from_dict）
    row_llm, res_llm = run_scenario_from_dict(
        scen["scenario_id"],
        scen,
        baseline_cost=baseline_cost,
        verbose=False,
    )

    llm_rows.append(row_llm)
    llm_details[scen["scenario_id"]] = {
        "raw_text": raw_text,
        "scenario_dict": scen,
        "or_result": res_llm,
    }

# ---------------------------------------------------------
# 4) 汇总所有 LLM 场景的对比结果
# ---------------------------------------------------------

if llm_rows:
    llm_summary_df = pd.DataFrame(llm_rows)

    print("\n=== 所有 LLM 场景 vs baseline 的成本对比 ===")
    cols = [
        "scenario_id",
        "summary_zh",
        "affected_center",
        "capacity_factor",
        "duration_days",
        "unmet_penalty",
        "total_cost",
        "extra_cost_vs_baseline",
        "total_unmet_t",
        "avg_utilization",
    ]
    cols = [c for c in cols if c in llm_summary_df.columns]
    display(llm_summary_df[cols])

    print("\n=== 简要列表：total_cost 和 extra_cost_vs_baseline ===")
    for _, r in llm_summary_df.iterrows():
        print(
            f"{r['scenario_id']}: "
            f"total_cost = {r['total_cost']:,.2f} AUD, "
            f"extra vs baseline = {r['extra_cost_vs_baseline']:,.2f} AUD"
        )
else:
    print("没有成功的 LLM 场景结果。")
    
rows_llm = []
# 所有 LLM 场景结果整理成 DataFrame
df_llm_cases = pd.DataFrame(rows_llm)
display(df_llm_cases)

# 和 exp_summary_all 合并保存
exp_summary_all = pd.concat([exp_summary_all, df_llm_cases], ignore_index=True)
exp_summary_all.to_csv(PROCESSED_DIR / "tbl_experiment_summary_all_scenarios.csv",
                       index=False)
print("已更新保存到 tbl_experiment_summary_all_scenarios.csv")

baseline_cost 已存在: 2,187,600.59 AUD

Baseline total cost: 2,187,600.59 AUD

[LLM_case_prompt_1] 调用 LLM ...


--- LLM 原始输出（前 400 字符）---

```json
{
  "scenario_id": "A",
  "summary_zh": "设施 Demo Recycling Center 1 (F01) 发生故障，未来5天处理能力降为40%。",
  "affected_center": "F01",
  "capacity_factor": 0.4,
  "duration_days": 5,
  "unmet_penalty": 10000
}
```...

--- 从 LLM 输出解析得到的 scenario_dict ---


{'scenario_id': 'A',
 'summary_zh': '设施 Demo Recycling Center 1 (F01) 发生故障，未来5天处理能力降为40%。',
 'affected_center': 'F01',
 'capacity_factor': 0.4,
 'duration_days': 5,
 'unmet_penalty': 10000.0,
 'raw_json': {'scenario_id': 'A',
  'summary_zh': '设施 Demo Recycling Center 1 (F01) 发生故障，未来5天处理能力降为40%。',
  'affected_center': 'F01',
  'capacity_factor': 0.4,
  'duration_days': 5,
  'unmet_penalty': 10000}}

[LLM_case_prompt_2] 调用 LLM ...


--- LLM 原始输出（前 400 字符）---

```json
{
  "scenario_id": "B",
  "summary_zh": "设施 Remanufacturing Plant 1 (F02) 因电力检修，未来10天只能维持60%的处理能力。",
  "affected_center": "F02",
  "capacity_factor": 0.6,
  "duration_days": 10,
  "unmet_penalty": 10000
}
```...

--- 从 LLM 输出解析得到的 scenario_dict ---


{'scenario_id': 'B',
 'summary_zh': '设施 Remanufacturing Plant 1 (F02) 因电力检修，未来10天只能维持60%的处理能力。',
 'affected_center': 'F02',
 'capacity_factor': 0.6,
 'duration_days': 10,
 'unmet_penalty': 10000.0,
 'raw_json': {'scenario_id': 'B',
  'summary_zh': '设施 Remanufacturing Plant 1 (F02) 因电力检修，未来10天只能维持60%的处理能力。',
  'affected_center': 'F02',
  'capacity_factor': 0.6,
  'duration_days': 10,
  'unmet_penalty': 10000}}

[LLM_case_prompt_3] 调用 LLM ...


--- LLM 原始输出（前 400 字符）---

```json
{
  "scenario_id": "C",
  "summary_zh": "F01 的进料道路出现交通拥堵，未来7天产能降至平时70%，需求未满足的惩罚成本提高至12000。",
  "affected_center": "F01",
  "capacity_factor": 0.7,
  "duration_days": 7,
  "unmet_penalty": 12000
}
```...

--- 从 LLM 输出解析得到的 scenario_dict ---


...


=== 所有 LLM 场景 vs baseline 的成本对比 ===


,scenario_id,summary_zh,affected_center,capacity_factor,duration_days,unmet_penalty,total_cost,extra_cost_vs_baseline,total_unmet_t,avg_utilization
A,A,设施 Demo Recycling Center 1 (F01) 发生故障，未来5天处理能力...,F01,0.4,5,10000.0,2.487547e+06,299946.218428,248.74349,0.666667
B,B,设施 Remanufacturing Plant 1 (F02) 因电力检修，未来10天只能...,F02,0.6,10,10000.0,2.301570e+06,113969.577068,230.14349,0.666667
C,C,F01 的进料道路出现交通拥堵，未来7天产能降至平时70%，需求未满足的惩罚成本提高至12000。,F01,0.7,7,12000.0,2.805061e+06,617460.088485,233.74349,0.666667



=== 简要列表：total_cost 和 extra_cost_vs_baseline ===
A: total_cost = 2,487,546.81 AUD, extra vs baseline = 299,946.22 AUD
B: total_cost = 2,301,570.17 AUD, extra vs baseline = 113,969.58 AUD
C: total_cost = 2,805,060.68 AUD, extra vs baseline = 617,460.09 AUD


""


已更新保存到 tbl_experiment_summary_all_scenarios.csv


In [13]:
# === 新 Cell：补齐 4 个 engineered 场景，并更新 exp_summary_all ===
import pandas as pd

# ---------------------------------------------------------
# 0. 重新定义 run_scenario_from_dict（关键：尊重 scenario_type）
# ---------------------------------------------------------
def run_scenario_from_dict(
    name: str,
    scenario_dict: dict,
    baseline_cost: float | None = None,
    verbose: bool = False,
):
    """
    根据一个 scenario_dict 组装 capacity_factor/unmet_penalty，
    调用 solve_bne_model 求解，并返回：
      - 一行 summary（Series）
      - 原始 res（solve_bne_model 的返回 dict）

    这里特别注意：scenario_type 从 scenario_dict 中读取，
    没写则默认 'LLM_generated'。
    """

    # 1) 基本字段
    sid            = scenario_dict.get("scenario_id", name)
    summary_zh     = scenario_dict.get("summary_zh", "")
    affected_center = scenario_dict.get("affected_center", None)
    scen_type      = scenario_dict.get("scenario_type", "LLM_generated")

    cap_factor     = float(scenario_dict.get("capacity_factor", 1.0))
    duration_days  = scenario_dict.get("duration_days", None)
    unmet_penalty  = float(scenario_dict.get("unmet_penalty", 10000))

    # 2) 组装 capacity_factor 映射
    fac_ids = list(facilities_df.index)
    cap_factor_dict = {fid: 1.0 for fid in fac_ids}

    if affected_center is not None:
        if affected_center not in fac_ids:
            raise RuntimeError(
                f"场景 {name} 中的 affected_center='{affected_center}' "
                f"不在设施列表中：{fac_ids}"
            )
        cap_factor_dict[affected_center] = cap_factor

    # 3) 调 OR / Gurobi 模型
    res = solve_bne_model(
        capacity_factor=cap_factor_dict,
        unmet_penalty=unmet_penalty,
        verbose=verbose,
    )

    total_cost  = res["total_cost"]
    total_unmet = res.get("total_unmet", None)

    # 4) 计算与 baseline 的成本差
    extra_cost = None
    if baseline_cost is not None and total_cost is not None:
        extra_cost = total_cost - baseline_cost

    # 5) 计算平均利用率
    avg_util = None
    util_df = res.get("util_df", None)
    if isinstance(util_df, pd.DataFrame) and "utilization" in util_df.columns:
        avg_util = util_df["utilization"].mean()

    # 6) 汇总成一行
    row = {
        "scenario_id":             sid,
        "scenario_type":           scen_type,
        "summary_zh":              summary_zh,
        "affected_center":         affected_center,
        "capacity_factor":         cap_factor,
        "duration_days":           duration_days,
        "unmet_penalty":           unmet_penalty,
        "total_cost":              total_cost,
        "total_unmet_t":           total_unmet,
        "extra_cost_vs_baseline":  extra_cost,
        "avg_utilization":         avg_util,
    }

    row_series = pd.Series(row, name=sid)
    return row_series, res


# ---------------------------------------------------------
# 1. 定义 4 个“工程化”场景（disruption + expansion 各 2 个）
# ---------------------------------------------------------
# 说明：
#  - ENG_F01_40pct_5days:  F01 严重故障，5 天只剩 40% 容量（已有，第 2 实验的 S1）
#  - ENG_F02_80pct_10days: F02 中度故障，10 天剩 80% 容量（已有，第 2 实验的 S2）
#  - ENG_F01_120pct_14days: F01 扩容 20%，持续 14 天（弹性提升场景）
#  - ENG_F02_60pct_3days:  F02 轻度短期故障，3 天剩 60% 容量（新补场景）

engineered_scenarios = [
    {
        "scenario_id":   "ENG_F01_40pct_5days",
        "scenario_type": "engineered_disruption",
        "summary_zh":    "F01 回收中心发生严重故障，未来 5 天容量仅为平时的 40%。",
        "affected_center": "F01",
        "capacity_factor": 0.4,
        "duration_days":  5,
        "unmet_penalty":  10000.0,
    },
    {
        "scenario_id":   "ENG_F02_80pct_10days",
        "scenario_type": "engineered_disruption",
        "summary_zh":    "F02 再制造工厂出现中度故障，未来 10 天容量降为 80%。",
        "affected_center": "F02",
        "capacity_factor": 0.8,
        "duration_days":  10,
        "unmet_penalty":  15000.0,
    },
    {
        "scenario_id":   "ENG_F01_120pct_14days",
        "scenario_type": "engineered_disruption",   # 这里可以在 paper 里强调是 “capacity expansion / resilience” 场景
        "summary_zh":    "F01 回收中心通过加班与临时工扩容，未来 14 天容量提升到 120%。",
        "affected_center": "F01",
        "capacity_factor": 1.2,
        "duration_days":  14,
        "unmet_penalty":  10000.0,
    },
    {
        "scenario_id":   "ENG_F02_60pct_3days",
        "scenario_type": "engineered_disruption",
        "summary_zh":    "F02 再制造工厂因短期检修，未来 3 天容量降为 60%。",
        "affected_center": "F02",
        "capacity_factor": 0.6,
        "duration_days":  3,
        "unmet_penalty":  15000.0,
    },
]


# ---------------------------------------------------------
# 2. 逐个调用 OR 模型求解 4 个 engineered 场景
# ---------------------------------------------------------
rows_eng = []

print("\n=== 求解 4 个 engineered 场景 ===")
for scen in engineered_scenarios:
    sid = scen["scenario_id"]
    print(f">>> Solving engineered scenario {sid} ...")
    row, res = run_scenario_from_dict(
        name=sid,
        scenario_dict=scen,
        baseline_cost=baseline_cost,  # 来自第二个实验或前面 cell
        verbose=False,
    )
    rows_eng.append(row)

df_eng = pd.DataFrame(rows_eng)

print("\n=== 新 engineered 场景求解结果（简表） ===")
cols_show = [
    "scenario_id", "scenario_type", "summary_zh",
    "affected_center", "capacity_factor", "duration_days",
    "total_cost", "extra_cost_vs_baseline"
]
display(df_eng[cols_show])


# ---------------------------------------------------------
# 3. 合并进 exp_summary_all 并保存
# ---------------------------------------------------------
# 确保 exp_summary_all 已经在内存（Cell1 / Cell7 前面已经加载过）
if "exp_summary_all" not in globals():
    exp_summary_path = PROCESSED_DIR / "tbl_experiment_summary_all_scenarios.csv"
    exp_summary_all = pd.read_csv(exp_summary_path)

# 避免重复：如果之前已经有同名 engineered 场景，就先删掉再追加
ids_new = df_eng["scenario_id"].tolist()
mask_old = (
    exp_summary_all["scenario_id"].isin(ids_new)
    & (exp_summary_all["scenario_type"] == "engineered_disruption")
)
exp_summary_all = exp_summary_all.loc[~mask_old].copy()

# 追加新结果
exp_summary_all = pd.concat([exp_summary_all, df_eng], ignore_index=True)

# 保存回 processed 目录
out_path = PROCESSED_DIR / "tbl_experiment_summary_all_scenarios.csv"
exp_summary_all.to_csv(out_path, index=False)
print(f"\n已更新并保存到 {out_path.name}")

# 简要统计，给你和审稿人一个“系统化”的感觉
scn_all = exp_summary_all[exp_summary_all["scenario_type"] != "baseline"].copy()

print("\n=== 更新后的场景统计（不含 baseline） ===")
print("总场景数:", len(scn_all))
print("\n按类型统计：")
print(scn_all["scenario_type"].value_counts())
print("\n按设施统计：")
print(scn_all["affected_center"].value_counts())



=== 求解 4 个 engineered 场景 ===
>>> Solving engineered scenario ENG_F01_40pct_5days ...
>>> Solving engineered scenario ENG_F02_80pct_10days ...
>>> Solving engineered scenario ENG_F01_120pct_14days ...
>>> Solving engineered scenario ENG_F02_60pct_3days ...

=== 新 engineered 场景求解结果（简表） ===


,scenario_id,scenario_type,summary_zh,affected_center,capacity_factor,duration_days,total_cost,extra_cost_vs_baseline
ENG_F01_40pct_5days,ENG_F01_40pct_5days,engineered_disruption,F01 回收中心发生严重故障，未来 5 天容量仅为平时的 40%。,F01,0.4,5,2.487547e+06,2.999462e+05
ENG_F02_80pct_10days,ENG_F02_80pct_10days,engineered_disruption,F02 再制造工厂出现中度故障，未来 10 天容量降为 80%。,F02,0.8,10,3.366803e+06,1.179202e+06
ENG_F01_120pct_14days,ENG_F01_120pct_14days,engineered_disruption,F01 回收中心通过加班与临时工扩容，未来 14 天容量提升到 120%。,F01,1.2,14,2.087619e+06,-9.998207e+04
ENG_F02_60pct_3days,ENG_F02_60pct_3days,engineered_disruption,F02 再制造工厂因短期检修，未来 3 天容量降为 60%。,F02,0.6,3,3.452288e+06,1.264687e+06



已更新并保存到 tbl_experiment_summary_all_scenarios.csv

=== 更新后的场景统计（不含 baseline） ===
总场景数: 23

按类型统计：
scenario_type
LLM_generated            16
engineered_disruption     7
Name: count, dtype: int64

按设施统计：
affected_center
F01    9
F02    8
F03    6
Name: count, dtype: int64


In [14]:
from pathlib import Path
import pandas as pd

# 统一路径
PROCESSED_DIR = Path(r"D:\Projects\llm_or_project\Notebook1\data\processed")

# 只读论文用的 FINAL 场景表
FINAL_SUMMARY_CSV = PROCESSED_DIR / "tbl_experiment_summary_FINAL.csv"

exp_summary_all = pd.read_csv(FINAL_SUMMARY_CSV)

print("当前使用的实验汇总文件:", FINAL_SUMMARY_CSV)
print("总行数(含 baseline)：", len(exp_summary_all))

print("\nscenario_type 统计：")
print(exp_summary_all["scenario_type"].value_counts())

print("\nscenario_id 列表：")
print(exp_summary_all["scenario_id"].tolist())

def save_experiment_results(df: pd.DataFrame, filename: str):
    """
    安全写入实验结果：禁止覆盖 FINAL CSV，只能写到其它文件。
    """
    target = PROCESSED_DIR / filename
    if target.name == FINAL_SUMMARY_CSV.name:
        raise RuntimeError("禁止覆盖 FINAL 场景表，请改用 tbl_experiment_summary_all_scenarios.csv 或其它文件名。")
    df.to_csv(target, index=False, encoding="utf-8-sig")
    print(f"✅ 已保存到: {target}")



当前使用的实验汇总文件: D:\Projects\llm_or_project\Notebook1\data\processed\tbl_experiment_summary_FINAL.csv
总行数(含 baseline)： 12

scenario_type 统计：
scenario_type
LLM_generated            6
engineered_disruption    5
baseline                 1
Name: count, dtype: int64

scenario_id 列表：
['BASE', 'F01_120pct_14days', 'F01_40pct_5days', 'F01_60pct_5days', 'F02_50pct_3days', 'F03_40pct_3days', 'LLM_case_F01_40pct_7days', 'LLM_case_F01_60pct_7days', 'LLM_case_F02_40pct_10days', 'LLM_case_F02_60pct_10days', 'LLM_case_F03_60pct_7days', 'LLM_case_F03_80pct_7days']
